In [2]:
import win32com.client.gencache as gc
import win32com
c = win32com.client.constants

After a couple hours of trying everything I could think of, the following series of commands are the ONLY way I could figure out how to get data from the DataSocket to update. Specifically, The correct ReadMode, AccessMode must be used, and you must NOT call DataSocket.Update(). 

Other attempts resulted in complete failure to obtain data, obtain data only on 1st call, and in a couple places stalled the working thread. 

In [3]:
ds = gc.EnsureDispatch("CWDSLib.CWDataSocket")
ds.ReadMode = c.cwdsReadsSynchronous        
ds.SyncConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate, -1)
ds.SyncRead(-1)

False

In [7]:
ds.SyncRead(-1)
ds.Data.Value

(('TempPV(C)',
  'TempAIsPrimaryActual',
  'TempHeatDutyActual(%)',
  'TempModeActual',
  'TempARaw(C)',
  'TempBRaw(C)',
  'InterlockHeater',
  'TempAIsActive',
  'TempBIsActive',
  'TempA(C)',
  'TempB(C)',
  'TempHeatDutyUser(%)',
  'TempSP(C)',
  'TempModeUser',
  'TempHardware',
  'TempUserConfig',
  'FilterOvenDutyUser(%)',
  'FilterOvenSP(C)',
  'Not Used 0',
  'AgModeActual',
  'AgPV(RPM)',
  'AgMainGasActualRequest(LPM)',
  'AgMainGasUser(LPM)',
  'AgModeUser',
  'AgSP(RPM)',
  'CalLimitsPressure',
  'SystemVars(XML)',
  'Flattened Bag String',
  'RIO Version',
  'BioreactorModel',
  'Cal Values',
  'LEDWhiteLEDOn',
  'pHAIsPrimaryActual',
  'pHBRaw',
  'pHARaw',
  'pHCO2ActualRequest(%)',
  'pHBaseDutyActual(%)',
  'pHA',
  'pHModeActual',
  'pHActiveMode',
  'pHAIsActive',
  'pHBIsActive',
  'pHPV',
  'pHB',
  'pHCO2User(%)',
  'pHBaseDutyUser(%)',
  'pHSP',
  'pHModeUser',
  'pHHardware',
  'pHUserConfig',
  'DOAIsPrimaryActual',
  'DOBRaw(%)',
  'DOARaw(%)',
  'DON2FlowAct

In [1]:
oldvalue = -1

In [21]:
ds.SyncRead(-1)
#ds.Update()
#ds.SyncRead(-1)
value = ds.Data.Value
print(value == oldvalue)
oldvalue = value
value

False


(-0.342020143325668,
 -0.37460659341591257,
 -0.4067366430758004,
 -0.43837114678907724,
 -0.46947156278589025,
 -0.49999999999999917,
 -0.5299192642332053,
 -0.5591929034707468,
 -0.5877852522924728,
 -0.6156614753256576,
 -0.6427876096865398,
 -0.6691306063588583,
 -0.6946583704589971,
 -0.7193398003386507,
 -0.7431448254773936,
 -0.7660444431189782,
 -0.7880107536067219,
 -0.8090169943749472,
 -0.8290375725550413,
 -0.8480480961564263,
 -0.8660254037844387,
 -0.8829475928589269,
 -0.8987940462991667,
 -0.9135454576426005,
 -0.9271838545667875,
 -0.9396926207859084,
 -0.9510565162951534,
 -0.9612616959383187,
 -0.9702957262759966,
 -0.9781476007338057,
 -0.984807753012208,
 -0.9902680687415703,
 -0.9945218953682732,
 -0.9975640502598243,
 -0.9993908270190958,
 -1.0,
 -0.9993908270190958,
 -0.9975640502598242,
 -0.9945218953682733,
 -0.9902680687415704,
 -0.9848077530122081,
 -0.9781476007338058,
 -0.9702957262759964,
 -0.9612616959383189,
 -0.9510565162951538,
 -0.9396926207859086,
 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib

In [ ]:
f = plt.figure()
ax = f.add_subplot(111)
line, = ax.plot(ds.Data.Value)
f.show()

In [ ]:
while True:
    ds.SyncRead(-1)
    y = ds.Data.Value
    x = list(range(len(y)))
    line.set_data(x, y)
    f.canvas.flush_events()
    f.canvas.draw()

In [ ]:
import win32com.client.gencache as gc
import win32com
c = win32com.client.constants

Just kidding, this way works too. 

http://zone.ni.com/reference/en-XX/help/375857A-01/html/t_nationalinstruments_net_readmode/
http://zone.ni.com/reference/en-XX/help/375857A-01/xml/usingdatasocketeventsaccessmodesnet/

tldr: in AutoUpdate access mode, the OnDataUpdated event is generated when data is changed. Otherwise, must use Update(). 
In ReadAsynchronous ReadMode, data is read automatically when updated from the server (aka pushed from server to client). Ohterwise, muse use SyncRead(). 

In [ ]:
class EventHandler:
    def OnStatusUpdated(self, *args):
        #print("OnStatusUpdate")
        pass
    def OnDataUpdated(self, ob):
        ob.Invoke(1, 0, pythoncom.DISPATCH_PROPERTYGET, 1)
        print("test")

ds = gc.EnsureDispatch("CWDSLib.CWDataSocket")
handler = win32com.client.WithEvents(ds, EventHandler)
ds.ReadMode = c.cwdsReadsAsynchronous        
# ds.SyncConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate, -1)
ds.ConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate)
#ds.SyncRead(-1)
oldvalue = -1

In [ ]:
#ds.SyncRead(-1)
#ds.Update()
#ds.SyncRead(-1)
value = ds.Data.Value
print(value == oldvalue)
oldvalue = value
value

In [ ]:
ds.Disconnect()

In [ ]:
dir(lastargs[0])

In [ ]:
lastarg = lastargs[0]
info = lastarg.GetTypeInfo()

In [ ]:
dir(info)

In [ ]:
info.GetNames(1)

In [ ]:
lastarg.Value

In [ ]:
import pythoncom

In [ ]:
pythoncom.DISPATCH_

In [ ]:
lastarg.Invoke(1, 0, pythoncom.DISPATCH_PROPERTYGET, 1)